In [1]:
# Run this cell first to load the necessary functions and libraries
from string import Template
import json


class mt(Template):
    delimiter = "&"
    idpattern = r"[a-z][_a-z0-9]*"


def dict_default(dict, para, default):
    return dict[para] if para in dict else default


def replace_string_in_script(script_content, search_string, replace_string):
    """
    Replace all occurrences of search_string with replace_string in the script_content.
    """
    return script_content.replace(search_string, replace_string)


def replace_line_with_string(script_content, search_string, replace_string):
    """
    Finds the first line containing search_string and replaces the entire line with replace_string.
    """
    lines = script_content.splitlines()
    for i, line in enumerate(lines):
        if search_string in line:
            lines[i] = replace_string
            break
    return "\n".join(lines)


def delete_lines_between(script_content, search_string1, search_string2):
    """
    Deletes all lines between (but not including) the lines which contain search_string1 and search_string2.
    """
    lines = script_content.splitlines()
    start_idx = end_idx = None
    for i, line in enumerate(lines):
        if start_idx is None and search_string1 in line:
            start_idx = i
        if start_idx is not None and end_idx is None and search_string2 in line:
            end_idx = i
            break
    if start_idx is not None and end_idx is not None and end_idx > start_idx + 1:
        del lines[start_idx + 1 : end_idx]
    return "\n".join(lines)

def replace_lines_between(script_content, search_string1, search_string2, replace_lines):
    """
    Deletes all lines between (but not including) the lines which contain search_string1 and search_string2.
    """
    lines = script_content.splitlines()
    start_idx = end_idx = None
    for i, line in enumerate(lines):
        if start_idx is None and search_string1 in line:
            start_idx = i
        if start_idx is not None and end_idx is None and search_string2 in line:
            end_idx = i
            break
    if start_idx is not None and end_idx is not None and end_idx > start_idx + 1:
        lines[start_idx + 1:end_idx] = replace_lines if isinstance(replace_lines, list) else [replace_lines]
    return "\n".join(lines)

def main(tcl_src_file_name,ip_name):

    tcl_file = "src/" + tcl_src_file_name
    helper_file = "helper.sh"


    with open(tcl_file, "r") as f:
        tcl_content = f.read()

    with open("tcl_templater.json", "r") as f:
        replace_dict = json.load(f)

    with open(helper_file, "r") as f:
        helper_content = f.read()

    for key, value in replace_dict["replace"].items():
        if isinstance(value, str):
            tcl_content = replace_line_with_string(tcl_content, key, value)

    for key, value in replace_dict["delete"].items():
        if isinstance(value, str):
            tcl_content = delete_lines_between(tcl_content, key, value)

    tcl_content = replace_string_in_script(tcl_content,ip_name+"_0","&{top}")
    tcl_content = replace_string_in_script(tcl_content,ip_name,"&{top}")
    tcl_content = replace_lines_between(tcl_content,"set script_file", "original project from",helper_content)



    with open(f"generated/templated_{tcl_src_file_name}", "w") as f:
        f.write(tcl_content)
    print(f"Modified TCL saved to ./generated/templated_{tcl_src_file_name}")



In [2]:
tcl_src_file_name = "new_hlx.tcl"  # Change this to your Vivado generated TCL
ip_name = "ACC"  # This is the name of the your custom IP in the Vivado block design without the "_0" at the end
main(tcl_src_file_name, ip_name)

Modified TCL saved to ./generated/templated_new_hlx.tcl
